### Biblioteca

In [41]:
#!pip install selenium
#!pip install webdriver_manager

In [42]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import base64
import pandas as pd
import requests
import time
import re
import numpy as np

from selenium.webdriver.common.keys import Keys

In [43]:
def custom_split(string):
        result = []
        start = 0
        while True:
            # Find the indices of '<' and '>'
            start_bracket = string.find('>', start)
            end_bracket = string.find('<', start)
            
            # If both '<' and '>' are found
            if start_bracket != -1 and end_bracket != -1:
                # Extract the substring between '<' and '>'
                result.append(string[start_bracket+1:end_bracket])
                # Update the start index for the next iteration
                start = end_bracket + 1
            else:
                break
        
        return result

def extract_value(tag, relation, dec_cont):
        result = dec_cont.find_all(tag)
        if not result:
            return np.nan
        
        value = result[0].text.strip()
        if relation == "float":
            return float(value) if value else np.nan
        elif relation == "bool":
            return True if value.lower() == "true" else False
        else:
            return value

def extract_float(tag, dec_cont):
        result = dec_cont.find_all(tag)
        if not result:
            return np.nan
        
        value = result[0].text.strip()
        return float(value) if value else np.nan

In [44]:
def relatorio_auto_fundos(cnpj_fundo):

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.set_page_load_timeout(60)
    cnpj_fundo = re.sub(r'\D', '', str(cnpj_fundo) )

    url = 'https://fnet.bmfbovespa.com.br/fnet/publico/abrirGerenciadorDocumentosCVM?cnpjFundo=' + cnpj_fundo
    driver.get(url)
    driver.implicitly_wait(10)
    WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.ID, 'showFiltros'))).click()

    WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.ID, 's2id_categoriaDocumento'))).click()
    driver.find_element(By.XPATH, '//*[@id="s2id_autogen2_search"]').send_keys("Informes Periódicos")
    driver.find_element(By.XPATH, '//*[@id="s2id_autogen2_search"]').send_keys(Keys.RETURN)

    time.sleep(4)

    WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="select2-chosen-3"]'))).click()
    driver.find_element(By.XPATH, '//*[@id="s2id_autogen3_search"]').send_keys("Informe Mensal Estruturado")
    driver.find_element(By.XPATH, '//*[@id="s2id_autogen3_search"]').send_keys(Keys.RETURN)

    time.sleep(4)

    WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.ID, 'filtrar'))).click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tblDocumentosEnviados_length"]/label/select'))).click()
    driver.find_element(By.XPATH, '//*[@id="tblDocumentosEnviados_length"]/label/select').send_keys("25")
    driver.find_element(By.XPATH, '//*[@id="tblDocumentosEnviados_length"]/label/select').send_keys(Keys.RETURN)

    # Datas
    data0 = pd.to_datetime(driver.find_element(By.XPATH, '//*[@id="tblDocumentosEnviados"]/tbody/tr[1]/td[5]').text)
    datas = [ data0 - pd.DateOffset(months = _) for _ in range(1, 13)]

    time.sleep(3)

    rows = driver.find_elements(By.CSS_SELECTOR, 'tr.odd')

    dados_serie12m = pd.DataFrame()

    # Iterate over the first 12 rows
    for row in rows[:12]:
        # Find the "Download do Documento" link within each row
        download_link = row.find_element(By.XPATH, './/a[@title="Download do Documento"]')
        
        # Get the URL of the download link
        download_url = download_link.get_attribute('href')
        response = requests.get(download_url)
        
        decoded_content = BeautifulSoup(base64.b64decode(response.content), "xml")

        dados_fundo = {
            # Registro
            "Nome do Fundo": extract_value("NomeFundo", "str", decoded_content),
            "CNPJ": extract_value("CNPJFundo", "str", decoded_content),
            "Data Inicio": extract_value("DataFuncionamento", "str", decoded_content),
            "Publico Alvo": extract_value("PublicoAlvo", "str", decoded_content),
            "Codigo ISIN": extract_value("CodigoISIN", "str", decoded_content),

            # Dados Registro
            "Numero de Cotas Emitidas": extract_float("QtdCotasEmitidas", decoded_content),
            "Fundo Exclusivo": extract_value("NomeFundoExclusivo", "bool", decoded_content),
            "Mandato": extract_value("Mandato", "str", decoded_content),
            "Segmento de Atuacao": extract_value("SegmentoAtuacao", "str", decoded_content),
            "Tipo de Gestao": extract_value("TipoGestao", "str", decoded_content),
            "Prazo": extract_value("PrazoDuracao", "str", decoded_content),
            "Data Encerramento": extract_value("DataPrazoDuracao", "str", decoded_content),
            "Encerramento do Exercicio": extract_value("EncerramentoExercicio", "str", decoded_content),
            "Nome Administrador": extract_value("NomeAdministrador", "str", decoded_content),
            "Data Competência": pd.to_datetime(extract_value("Competencia", "str", decoded_content), format= r'%Y/%m/%d'),

            # Dados Cotistas
            "Cotistas Total": extract_float("PessoaFisica", decoded_content) + extract_float("PJNaoFinanceira", decoded_content) + extract_float("BancoComercial", decoded_content) +
                            extract_float("CorretoraDistribuidora", decoded_content) + extract_float("OutrasPJFinanceiras", decoded_content) +
                            extract_float("InvestidoresNaoResidentes", decoded_content) + extract_float("EntidadeAbertaPrevCompl", decoded_content) +
                            extract_float("EntidadeFechadaPrevCompl", decoded_content) + extract_float("FundosInvImobiliario", decoded_content) +
                            extract_float("OutrosFundosInv", decoded_content),
            "Cotistas PF": extract_float("PessoaFisica", decoded_content), 
            "Cotistas PJ": extract_float("PJNaoFinanceira", decoded_content),
            "Banco Comercial": extract_float("BancoComercial", decoded_content),
            "Corretora Distribuidora": extract_float("CorretoraDistribuidora", decoded_content),
            "Outras PJ Financeiras": extract_float("OutrasPJFinanceiras", decoded_content),
            "Investidores Nao Residentes": extract_float("InvestidoresNaoResidentes", decoded_content),
            "Entidades Abertas Previdencia Complementar": extract_float("EntidadeAbertaPrevCompl", decoded_content),
            "Entidades Fechada Previdencia Complementar": extract_float("EntidadeFechadaPrevCompl", decoded_content),
            "Fundos de Investimentos Imobiliarios": extract_float("FundosInvImobiliario", decoded_content),
            "Outros Fundos de Investimentos": extract_float("OutrosFundosInv", decoded_content),

            # Resumo
            "Ativo": extract_float("Ativo", decoded_content),
            "Patrimonio Liquido": extract_float("PatrimonioLiquido", decoded_content),
            "Numero de cotas emitidas": extract_float("NumCotasEmitidas", decoded_content),
            "Valor Patrimonial Cota": extract_float("ValorPatrCota", decoded_content),
            "Despesas Taxa Administracao": extract_float("DespesasTxAdministracao", decoded_content),
            "Despesas Agente Custodiante": extract_float("DespesasAgCustodiante", decoded_content),

            # Rentabilidade
            "Rentabilidade Efetiva Mes": extract_float("RentPatrimonialMes", decoded_content) + extract_float("DividendYieldMes", decoded_content),
            "Rentabilidade Patrimonial Mes": extract_float("RentPatrimonialMes", decoded_content),
            "Dividend Yield Mes": extract_float("DividendYieldMes", decoded_content),

            # Ativo
            "Total Necessidades Liq": extract_float("Disponibilidades", decoded_content) + extract_float("TitulosPublicos", decoded_content) + extract_float("TitulosPrivados", decoded_content) + extract_float("FundosRendaFixa", decoded_content),
            "Disponibilidades": extract_float("Disponibilidades", decoded_content),
            "Titulos Publicos": extract_float("TitulosPublicos", decoded_content),
            "Titulos Privados": extract_float("TitulosPrivados", decoded_content),
            "Fundos RendaFixa": extract_float("FundosRendaFixa", decoded_content),
            "Total Investido": extract_float("Terrenos", decoded_content) + extract_float("ImoveisRendaAcabados", decoded_content) + extract_float("ImoveisRendaConstrucao", decoded_content) + extract_float("OutrosDireitosReais", decoded_content) + extract_float("Acoes", decoded_content) + extract_float("Debentures", decoded_content) + extract_float("BonusSubscricao", decoded_content) + extract_float("CertificadosDepositoValMob", decoded_content) + extract_float("CedulasDebentures", decoded_content)+extract_float("FIA", decoded_content)+extract_float("FIP", decoded_content)+extract_float("FII", decoded_content)+extract_float("FIDC", decoded_content)+extract_float("OutrasCotasFI", decoded_content)+extract_float("NotasPromissorias", decoded_content)+extract_float("AcoesSociedadesAtivFII", decoded_content)+extract_float("CotasSociedadesAtivFII", decoded_content)+extract_float("CEPAC", decoded_content)+extract_float("CriCra", decoded_content)+extract_float("LetrasHipotecarias", decoded_content)+extract_float("LciLca", decoded_content)+extract_float("OutrosValoresMobliarios", decoded_content),
            "Direitos Bens Imoveis": extract_float("Terrenos", decoded_content) + extract_float("ImoveisRendaAcabados", decoded_content) + extract_float("ImoveisRendaConstrucao", decoded_content) + extract_float("OutrosDireitosReais", decoded_content),
            "Terrenos": extract_float("Terrenos", decoded_content),
            "Imoveis Renda Acabados": extract_float("ImoveisRendaAcabados", decoded_content),
            "Imoveis Renda Construcao": extract_float("ImoveisRendaConstrucao", decoded_content),
            "Outros Direitos Reais": extract_float("OutrosDireitosReais", decoded_content),
            "Acoes": extract_float("Acoes", decoded_content),
            "Debentures": extract_float("Debentures", decoded_content),
            "Bonus de Subscricao": extract_float("BonusSubscricao", decoded_content),
            "Certificados Deposito Val Mob": extract_float("CertificadosDepositoValMob", decoded_content),
            "Cedulas Debentures": extract_float("CedulasDebentures", decoded_content),
            "FIA": extract_float("FIA", decoded_content),
            "FIP": extract_float("FIP", decoded_content),
            "FII": extract_float("FII", decoded_content),
            "FIDC": extract_float("FIDC", decoded_content),
            "Outras Cotas FI": extract_float("OutrasCotasFI", decoded_content),
            "Notas Promissorias": extract_float("NotasPromissorias", decoded_content),
            "Acoes Sociedades Ativ FII": extract_float("AcoesSociedadesAtivFII", decoded_content),
            "Cotas Sociedades Ativ FII": extract_float("CotasSociedadesAtivFII", decoded_content),
            "CEPAC": extract_float("CEPAC", decoded_content),
            "CRI/CRA": extract_float("CriCra", decoded_content),
            "Letras Hipotecarias": extract_float("LetrasHipotecarias", decoded_content),
            "LCI/LCA": extract_float("LciLca", decoded_content),
            "LIG": extract_float("LIG", decoded_content),
            "Outros Valores Mobiliarios": extract_float("OutrosValoresMobliarios", decoded_content),
            "Valores a Receber": extract_float("Alugueis", decoded_content) + extract_float("VendaImoveis", decoded_content) + extract_float("OutrosValores", decoded_content),
            "Alugueis": extract_float("Alugueis", decoded_content),
            "Venda Imoveis": extract_float("VendaImoveis", decoded_content),
            "Outros Valores": extract_float("OutrosValores", decoded_content),
            # Passivo
            "Rendimentos a Distribuir": extract_float("RendimentosDistribuir", decoded_content),
            "Taxa de Administracao a Pagar": extract_float("TxAdministracaoPagar", decoded_content),
            "Taxa de Performance a Pagar": extract_float("TxPerformancePagar", decoded_content),
            "Obrigacoes Aquisicao de Imovel": extract_float("ObrigacoesAquisicaoImov", decoded_content),
            "Adiantamento Venda Imovel": extract_float("AdiantamentoVendaImov", decoded_content),
            "Adiantamento de Alugueis": extract_float("AdiantamentoAlugueis", decoded_content),
            "Obrigacoes SecRecebiveis": extract_float("ObrigacoesSecRecebiveis", decoded_content),
            "Instrumentos Financeiros e Derivativos": extract_float("InstrumentosFinanceirosDeriv", decoded_content),
            "Provisoes Contigencias": extract_float("ProvisoesContigencias", decoded_content),
            "Outros Valores a Pagar": extract_float("OutrosValoresPagar", decoded_content),
            "Total Passivo": extract_float("TotalPassivo", decoded_content)
        }

        dados_serie12m = pd.concat([dados_serie12m, pd.DataFrame(dados_fundo, index = [dados_fundo["Encerramento do Exercicio"]])], axis = 0)

    dados_serie12m.index = datas

    dados_series = dados_serie12m.T

    dados_series.loc["Rentabilidade Efetiva Mes"] = dados_series.loc[["Rentabilidade Patrimonial Mes", "Dividend Yield Mes"]].sum()
    dados_series.loc["Cotistas Total"] = dados_series.loc[["Cotistas PF", "Cotistas PJ", "Banco Comercial", "Corretora Distribuidora", "Outras PJ Financeiras", 
                                                    "Investidores Nao Residentes", "Entidades Abertas Previdencia Complementar", "Entidades Fechada Previdencia Complementar", "Fundos de Investimentos Imobiliarios", "Outros Fundos de Investimentos"]].sum()
    dados_series.loc["Total Necessidades Liq"] = dados_series.loc[["Disponibilidades", "Titulos Publicos", "Titulos Privados", "Fundos RendaFixa"]].sum()
    dados_series.loc["Direitos Bens Imoveis"] = dados_series.loc[["Terrenos", "Imoveis Renda Acabados", "Imoveis Renda Construcao", "Outros Direitos Reais"]].sum()
    dados_series.loc["Total Investido"] = dados_series.loc[["Direitos Bens Imoveis", "Acoes", "Debentures", "Bonus de Subscricao", "Certificados Deposito Val Mob", "Cedulas Debentures", 
                                            "FIA", "FIP", "FII", "FIDC", "Outras Cotas FI", "Notas Promissorias", "Acoes Sociedades Ativ FII", "Cotas Sociedades Ativ FII", 
                                            "CEPAC", "CRI/CRA", "Letras Hipotecarias", "LCI/LCA", "Outros Valores Mobiliarios"]].sum()
    dados_series.loc["Valores a Receber"] = dados_series.loc[["Alugueis", "Venda Imoveis", "Outros Valores"]].sum()


    df_registros = ["Nome do Fundo", "CNPJ", "Data Inicio", "Publico Alvo", "Codigo ISIN"]
    df_resumo_balanco = ["Ativo", "Patrimonio Liquido", "Total Passivo"]
    df_ativos = ["Ativo", "Patrimonio Liquido", "Total Passivo", "Total Necessidades Liq", "Total Investido", "Direitos Bens Imoveis",
            "Disponibilidades", "Titulos Publicos", "Titulos Privados", "Fundos RendaFixa", "Terrenos", "Imoveis Renda Acabados", "Imoveis Renda Construcao", "Outros Direitos Reais",
            "Acoes", "Debentures", "Bonus de Subscricao", "Certificados Deposito Val Mob", "Cedulas Debentures", "FIA", "FIP", "FII", "FDIC", "Outras Cotas FI", "Notas Promissorias", "Acoes Sociedades Ativ FII", "Cotas Sociedades Ativ FII", "CEPAC",
            "CRI/CRA", "Letras Hipotecarias", "LCI/LCA", "LIG", "Outros Valores Mobliarios", "Valores a Receber","Alugueis", "Venda Imoveis", "Outros Valores"]
    df_rend = ["Patrimonio Liquido", "Rendimentos a Distribuir"]
    df_passivo = ["Total Passivo", "Taxa de Administracao a Pagar", "Taxa de Performance a Pagar", "Obrigacoes Aquisicao de Imovel", "Adiantamento Venda Imovel", "Obrigacoes SecRecebiveis", "Instrumentos Financeiros e Derivativos",
            "Provisoes Contigencias", "Outros Valores a Pagar"]


    dividendyield_12m = dados_series.loc["Dividend Yield Mes"].sum()
    rentabilidade_efetiva12m = dados_series.loc["Rentabilidade Efetiva Mes"].add(1).prod() - 1
    pl_cota = dados_series.loc["Patrimonio Liquido"] / dados_series.loc["Numero de cotas emitidas"]
    valores_a_receber = dados_series.loc[["Alugueis", "Venda Imoveis", "Outros Valores"]]
    rendimentos_a_distribuir_cota = dados_series.loc["Rendimentos a Distribuir"] / dados_series.loc["Numero de cotas emitidas"]
    alocacao = dados_series.loc[["Direitos Bens Imoveis", "Acoes", "Debentures", "Bonus de Subscricao", "Certificados Deposito Val Mob", "Cedulas Debentures", 
                                                "FIA", "FIP", "FII", "FIDC", "Outras Cotas FI", "Notas Promissorias", "Acoes Sociedades Ativ FII", "Cotas Sociedades Ativ FII", 
                                                "CEPAC", "CRI/CRA", "Letras Hipotecarias", "LCI/LCA", "Outros Valores Mobiliarios"]]

    return dados_series

In [45]:
cnpj_fundo = "29.641.226/0001-53"

In [46]:
dados_series = relatorio_auto_fundos(cnpj_fundo)

C:\Users\Titom\AppData\Local\Temp\ipykernel_2020\3628420887.py:30: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data0 = pd.to_datetime(driver.find_element(By.XPATH, '//*[@id="tblDocumentosEnviados"]/tbody/tr[1]/td[5]').text)


In [47]:
dados_series.loc["Rentabilidade Efetiva Mes"].add(1).prod() - 1

0.1850302764482452

In [48]:
dados_series.loc["Rentabilidade Efetiva Mes"].add(1).prod() - 1

0.1850302764482452